# Electra experimenty

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import tensorflow as tf
from torch.utils.data import Dataset
import re
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, recall_score, precision_score, f1_score
from sklearn.preprocessing import LabelEncoder
from typing import Dict, List
import transformers
from transformers import ElectraModel, ElectraConfig, ElectraTokenizer, ElectraForSequenceClassification
from transformers import Trainer, TrainingArguments


In [ ]:
# !pip install transformers[torch]
# !pip install accelerate -U
# !pip install transformers==4.30

## Načítanie a príprava dát

In [ ]:
dataset = pd.read_csv('../Data/final_dataset_3.csv')

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(dataset['processed_text'], dataset['author_id'], test_size=0.2, random_state=42)
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.1, random_state=42)

In [ ]:
label_encoder = LabelEncoder()

y_train = label_encoder.fit_transform(y_train)
y_valid = label_encoder.transform(y_valid)
y_test = label_encoder.transform(y_test)

## Electra

In [ ]:
class CustomDataset(Dataset):

    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

In [ ]:
def compute_metrics(p):
    pred, labels = p
    pred = np.argmax(pred, axis=1)

    accuracy = accuracy_score(y_true=labels, y_pred=pred)
    recall = recall_score(y_true=labels, y_pred=pred, average='macro', zero_division=0)
    precision = precision_score(y_true=labels, y_pred=pred, average='macro', zero_division=0)
    f1 = f1_score(y_true=labels, y_pred=pred, average='macro')

    return {
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1": f1
    }

In [ ]:
x_train = list(X_train)
x_val = list(X_valid)
x_test = list(X_test)

y_train = list(y_train)
y_val = list(y_valid)
y_test = list(y_test)

In [ ]:
tokenizer = ElectraTokenizer.from_pretrained('google/electra-small-discriminator')
train_encodings = tokenizer(x_train, truncation=True, padding=True)
val_encodings = tokenizer(x_val,truncation=True, padding=True)
test_encodings = tokenizer(x_test, truncation=True, padding=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

In [ ]:
train_dataset = CustomDataset(train_encodings, y_train)
val_dataset = CustomDataset(val_encodings, y_valid)
test_dataset = CustomDataset(test_encodings, y_test)

In [ ]:
labels_count = len(dataset['author_id'].unique())
print(labels_count)

20


## Experiment 1 - learning_rate = 5e-5

In [ ]:
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=15,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    weight_decay=0.01,
    learning_rate=5e-5,             #default
    logging_dir='./logs',
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    greater_is_better=True,
    fp16=True,
)

model = ElectraForSequenceClassification.from_pretrained("google/electra-small-discriminator", num_labels=labels_count)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
)

trainer.train()

pytorch_model.bin:   0%|          | 0.00/54.2M [00:00<?, ?B/s]

Some weights of the model checkpoint at google/electra-small-discriminator were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense.weight', 'discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense.bias']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at google/electra-small-discriminator and are newly initialized: ['classifier

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,No log,2.879720,0.160256,0.064719,0.121429,0.055658
2,No log,2.616856,0.224359,0.178947,0.202440,0.146085
3,2.717800,2.359394,0.282051,0.129075,0.256667,0.155378
4,2.717800,2.094398,0.378205,0.240842,0.345253,0.258315
5,2.717800,1.863860,0.429487,0.312080,0.391999,0.315317
6,1.896900,1.669158,0.512821,0.375663,0.463748,0.391353
7,1.896900,1.535325,0.544872,0.377510,0.495409,0.411645
8,1.896900,1.411926,0.570513,0.449691,0.575565,0.488603
9,1.241500,1.340065,0.608974,0.524099,0.605063,0.522588
10,1.241500,1.248631,0.608974,0.548222,0.626200,0.555758


TrainOutput(global_step=2640, training_loss=1.3946633021036783, metrics={'train_runtime': 288.2313, 'train_samples_per_second': 73.066, 'train_steps_per_second': 9.159, 'total_flos': 619876921098240.0, 'train_loss': 1.3946633021036783, 'epoch': 15.0})

In [ ]:
trainer.predict(test_dataset)

PredictionOutput(predictions=array([[-3.9394531 , -1.1748047 , -2.0253906 , ..., -1.9140625 ,
         4.5273438 , -0.19262695],
       [-1.0302734 ,  2.8085938 ,  2.1640625 , ...,  0.8359375 ,
        -0.8642578 , -3.3828125 ],
       [ 0.24597168,  2.8398438 ,  2.6015625 , ...,  1.390625  ,
        -1.6992188 , -3.0332031 ],
       ...,
       [-3.2753906 , -0.13549805, -0.6621094 , ..., -0.23095703,
         0.92285156, -2.828125  ],
       [-0.3852539 , -2.8789062 , -3.0976562 , ..., -1.5185547 ,
         0.5913086 ,  4.0585938 ],
       [-0.62353516, -1.71875   , -2.2890625 , ..., -1.1572266 ,
         1.8769531 ,  2.2285156 ]], dtype=float32), label_ids=array([18,  1,  2, 18, 15,  9,  1, 19,  6, 15, 15, 18,  1, 12, 14,  8,  4,
       19,  9, 16, 17, 16,  7, 13,  0,  7, 15, 15, 11,  4, 17, 12,  5, 18,
       12,  7, 16,  1, 17, 19,  2,  8,  4,  2, 19,  7,  7,  7,  8,  5,  7,
       11,  8,  5, 10,  9,  9, 19, 19,  8,  2, 12, 12, 19,  6, 12, 15,  0,
       18, 12,  6,  8, 12, 16,  

## Experiment 4 - learning_rate = 8e-5

In [ ]:
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=12,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    warmup_steps=300,
    weight_decay=0.01,
    learning_rate=8e-5,
    logging_dir='./logs',
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    greater_is_better=True,
    fp16=True,
)

model = ElectraForSequenceClassification.from_pretrained("google/electra-small-discriminator", num_labels=labels_count)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
)

trainer.train()

Some weights of the model checkpoint at google/electra-small-discriminator were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense.weight', 'discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense.bias']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at google/electra-small-discriminator and are newly initialized: ['classifier

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,No log,2.969839,0.096154,0.004808,0.050000,0.008772
2,No log,2.878255,0.147436,0.047770,0.118132,0.064297
3,No log,2.648475,0.237179,0.117449,0.205034,0.136838
4,No log,2.259446,0.416667,0.281972,0.402553,0.280282
5,No log,2.052465,0.435897,0.331548,0.413972,0.322898
6,2.505700,1.753887,0.506410,0.363752,0.490129,0.368514
7,2.505700,1.582033,0.557692,0.473906,0.545545,0.470725
8,2.505700,1.499341,0.589744,0.534176,0.587578,0.498855
9,2.505700,1.352907,0.596154,0.518346,0.583277,0.513258
10,2.505700,1.299888,0.634615,0.621879,0.621906,0.588398


TrainOutput(global_step=1056, training_loss=1.7566927635308467, metrics={'train_runtime': 200.7971, 'train_samples_per_second': 83.906, 'train_steps_per_second': 5.259, 'total_flos': 495901536878592.0, 'train_loss': 1.7566927635308467, 'epoch': 12.0})

In [ ]:
trainer.predict(test_dataset)

PredictionOutput(predictions=array([[-1.5693359 ,  0.20202637, -1.2753906 , ..., -1.7080078 ,
         3.2109375 ,  0.0340271 ],
       [ 0.3876953 ,  2.2890625 ,  1.6142578 , ...,  0.6982422 ,
        -1.3691406 , -1.9453125 ],
       [ 0.11346436,  2.5273438 ,  1.5292969 , ...,  0.67871094,
        -1.3554688 , -2.1582031 ],
       ...,
       [-1.6914062 ,  0.30517578, -1.21875   , ..., -1.5332031 ,
         3.1699219 , -0.2861328 ],
       [-1.2841797 , -1.7314453 , -2.1308594 , ..., -1.5166016 ,
         2.0839844 ,  3.3378906 ],
       [-0.66796875, -0.9555664 , -1.6943359 , ..., -0.97558594,
         2.9941406 ,  1.5498047 ]], dtype=float32), label_ids=array([18,  1,  2, 18, 15,  9,  1, 19,  6, 15, 15, 18,  1, 12, 14,  8,  4,
       19,  9, 16, 17, 16,  7, 13,  0,  7, 15, 15, 11,  4, 17, 12,  5, 18,
       12,  7, 16,  1, 17, 19,  2,  8,  4,  2, 19,  7,  7,  7,  8,  5,  7,
       11,  8,  5, 10,  9,  9, 19, 19,  8,  2, 12, 12, 19,  6, 12, 15,  0,
       18, 12,  6,  8, 12, 16,  